# Linear MDPの別のClippingについて

[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)では価値をクリップする方法で解きました．

---

**復習**

$$
\begin{aligned}
& b_h^n(s, a)=\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}\\
& \widehat{Q}_h^n(s, a)=\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\left(\theta^{\star}+\left(\widehat{\mu}_h^n\right)^{\top} \widehat{V}_{h+1}^n\right)^{\top} \phi(s, a) \\
& \widehat{V}_h^n(s)=\min \left\{\max _a \widehat{Q}_h^n(s, a), H\right\}, \quad \pi_h^n(s)=\operatorname{argmax}_a \widehat{Q}_h^n(s, a)
\end{aligned}
$$

このとき，次のOptimismが成立しています：
$$
\begin{aligned}
\hat{Q}_h^n(s, a)-Q_h^{\star}(s, a) 
\geq 
\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\phi(s, a)^{\top}\left(\widehat{\mu}_h^n-\mu_h^{\star}\right)^{\top} \widehat{V}_{h+1}^n
\geq 0
\end{aligned}
$$
が言えます．ここで最後の部分では
$$
\begin{aligned}
&\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widehat{V}_{h+1}^n\right| 
\overset{(a)}{\leq}
\sup_{V \in \mathcal{F}}\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot V \right| 
\lesssim \beta\|\phi(s, a)\|_{\left(\Lambda_h^n\right)^{-1}}\\
&\mathcal{F}=\left\{f_{w, \beta, \Lambda}:\|w\|_2 \leq L, \beta \in[0, B], \sigma_{\min }(\Lambda) \geq \lambda\right\}
\end{aligned}
$$

を使っています．(a)はcovering argumentです．

---

別のクリップだと深いことが言える気がしたので，メモしておきます．

表記：[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)と同じ

## 失敗ケース

私は最初に「$\widehat{P}^n_h$が確率分布になっていないので，$\widehat{P}^n_h$を確率単体に射影すればうまくいくのでは？」と思いましたが，雑にやるとうまくいきません．
例として，次を考えてみましょう：

$$
\begin{aligned}
v^{\min}_{h+1} =& \min_{s'}\widehat{V}_{h+1}^n(s')\; \text{ and }\; v^{\max}_{h+1} = \max_{s'}\widehat{V}_{h+1}^n(s')\\
 \widehat{Q}_h^n(s, a)=&
\beta_h \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)} + 
r(s, a)
+ \operatorname{clip}\left\{\widehat{P}^n_h(\cdot \mid s, a) \widehat{V}_{h+1}^n,\; v^{\min}_{h+1},\; v^{\max}_{h+1}\right\}
\end{aligned}
$$

これは$\widehat{P}^n_h$の変わりに，次の射影によって得られた遷移を使っているのと同じです：

$$
\widetilde{P}^n_h(\cdot \mid s, a) = 
\argmin_{p \in \Delta(\mathcal{S})}
|p^\top \widehat{V}^n_{h+1} - \widehat{P}^n_h(\cdot \mid s, a) \widehat{V}^n_{h+1}|\;.
$$
なぜなら，
$$\left\{p^\top \widehat{V}^n_{h+1}\mid p \in \Delta(\mathcal{S})\right\} = \left\{c \mid c \in [\min_{s'}\widehat{V}^n_{h+1}(s'), \max_{s'}\widehat{V}^n_{h+1}(s')]\right\}$$
と同じだからですね．

**よって，上のClippingをすると，$\widetilde{P}^n_h$上で，さらにボーナスが追加されたモデルベースRLを解いていることになります．**
ここで，この$\widetilde{P}_h$について，
$$
\begin{aligned}
&\left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq \left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-\widehat{P}^n_h(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| + \left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq 2\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\overset{(a)}{\leq}
2\sup_{V \in \widetilde{\mathcal{F}}}\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \underbrace{V}_{\leq (H-h)(1+\beta_{h+1})} \right| 
% \lesssim 
% \beta\|\phi(s, a)\|_{\left(\Lambda_h^n\right)^{-1}}\\
% &\widetilde{\mathcal{F}}=\left\{f_{w, \beta, \Lambda}:\|w\|_2 \leq L, \beta \in[0, B], \sigma_{\min }(\Lambda) \geq \lambda\right\}
\end{aligned}
$$
が成り立っています．

このアルゴリズムについて，$h+1$から$h$に降りる時を考えると，
$\widetilde{V}^n_{h+1}\propto b^n_{h+1}$なのに，さらに$b^n_h$倍のスケールが$\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right)\right|$のせいで出てきてしまいます．
つまり，**リグレット解析でボーナスの影響が$^H$乗されて出てきてしまい，うまくいきません．**


## 別のClipping

上とは少し違って，次のClippingはうまくいきます：
$$
\begin{aligned}
& \widehat{Q}_h^n(s, a)=r_h(s, a) + \min\left\{b^n_h(s, a)+\widehat{P}_h^n (\cdot \mid s, a)\widehat{V}_{h+1}^n(\cdot), H-h\right\} \\
& \widehat{V}_h^n(s)=\max _a \widehat{Q}_h^n(s, a), \quad \pi_h^n(s)=\operatorname{argmax}_a \widehat{Q}_h^n(s, a)
\end{aligned}
$$

---

**コメント**

そもそもどんなOptimismが欲しいのか思い出しましょう．
我々は，遷移の推定$\widehat{P}^n_h$と，任意の$V\in \mathcal{F}$について，$\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot V\right| \leq b^n_h$が高確率で成り立つことを知っています．

この事実を使って，任意の方策$\pi$について，

$$
\begin{aligned}
\hat{Q}_h^\pi(s, a) \geq r_h(s, a) + P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)
\end{aligned}
$$
が成り立つように何らかの$\widehat{Q}$を設計すれば，value difference lemmaからOptimismを示せます．

ここで，

$$
\widetilde{P}_h = \argmax_{p \in \Delta(\mathcal{S})} p^\top \widehat{V}^n_{h+1}
\text{ such that } |p^\top \widehat{V}^n_{h+1} - \widehat{P}^n_h(\cdot \mid s, a) \widehat{V}^n_{h+1}| \leq b^n_h(s, a)\;.
$$

なる$\widetilde{P}_h$を考えてみましょう．実行可能解に$P$が高確率で入っているので，この最適化問題は解を持ちます．
これはつまり，$\widehat{P}\widehat{V}$とあまり離れないValidな遷移のうち，最もOptimisticなものを返しています．よって，（適切な$\widehat{V}^n_{h+1}$とそのCoveringの$\mathcal{F}$）を考えると，

$$
\begin{aligned}
\widehat{P}_h^n (\cdot \mid s, a)\widehat{V}_{h+1}^n(\cdot) + b^n_h(s, a)
\geq
\widetilde{P}_h^n (\cdot \mid s, a)\widehat{V}_{h+1}^n(\cdot)
\geq 
P_h^n (\cdot \mid s, a)\widehat{V}_{h+1}^n(\cdot)
\end{aligned}
$$

が言えます．さらに両辺を$\min\{\cdot, H\}$すれば，アルゴリズムでOptimismが成り立つことがすぐにわかります．

---

**Validな遷移について**

これはClipしているのでValidなMDPを考えることができない…と思ってましたが，次のようにしてMDPを考えることができます：

$$
\begin{aligned}
\hat{Q}_h^\pi(s, a) 
&= r_h(s, a) + \min \left\{b^n_h(s, a) + \widehat{P}^n_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot), H - h\right\}\\
&\geq r_h(s, a) + P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)
\end{aligned}
$$
なので，
$$
\begin{aligned}
2b^n_h(s, a)
&\geq b^n_h(s, a) + \left|\widehat{P}^n_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot) - P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)\right|\\
&\geq b^n_h(s, a) + \widehat{P}^n_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot) - P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)\\
&\geq \min \left\{b^n_h(s, a) + P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot), H - h\right\} - P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)\\
&=: \delta^n_h(s, a) \geq 0
\end{aligned}
$$
なる非負の報酬$\delta^n_h(s, a)$を使うと，
$$
\begin{aligned}
\hat{Q}_h^\pi(s, a) 
&= r_h(s, a) + \min \left\{b^n_h(s, a) + \widehat{P}^n_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot), H - h\right\}\\
&= r_h(s, a) + \delta^n_h(s, a) + P_h(\cdot \mid s, a) \hat{V}^{\pi}_{h+1}(\cdot)
\end{aligned}
$$
なるValidなMDPを解いているのと同じです．そして，$\delta^n_h\geq 0$ですが，$\delta^n_h$はだんだん小さくなるため，徐々に最適方策に近づくことが言えるわけですね．

**注意**：この$\delta$は方策$\pi$に依存することに注意．


## 応用

CMDPのラグランジュ法で出てきますが，次のようなclippingの組み合わせを考えてみましょう．

$$
\begin{aligned}
&\widehat{Q}^{o, \pi}_h(s, a)=r^o_h(s, a) + \min\left\{b_h(s, a)+\widehat{P}_h (\cdot \mid s, a)\widehat{V}^{o, \pi}_{h+1}(\cdot), H-h\right\}\\
&\widehat{Q}^{p, \pi}_h(s, a)=r^p_h(s, a) + \max\left\{-b_h(s, a)+\widehat{P}_h (\cdot \mid s, a)\widehat{V}^{p, \pi}_{h+1}(\cdot), 0\right\} \\
&\widehat{V}^{o, \pi}_h(s) = \sum_{a} \pi_h(s) \widehat{Q}^{o, \pi}_h(s, a)
\quad \widehat{V}^{p, \pi}_h(s) = \sum_{a} \pi_h(s) \widehat{Q}^{p, \pi}_h(s, a)
\end{aligned}
$$

ここで，$h=H$では
$$
\begin{aligned}
\widetilde{\pi}_H(s) &= \argmax_{a} \widehat{Q}^{o, \widetilde{\pi}}_H(s, a) + \lambda \widehat{Q}^{p, \widetilde{\pi}}_H(s, a)\\
&= \argmax_{a} r^o_H(s, a) + r^p_H(s, a)
\end{aligned}
$$

とし，それ以外では

$$
\begin{aligned}
\widetilde{\pi}_h(s) &= \argmax_{a} \widehat{Q}^{o, \widetilde{\pi}}_h(s, a) + \lambda \widehat{Q}^{p, \widetilde{\pi}}_h(s, a)
\end{aligned}
$$
と計算した方策を考えます．
ここで，$\widetilde{\pi}$はClippingのせいで
$
\widehat{V}^{o, \pi}_1(s_1) + \lambda \widehat{V}^{p, \pi}_1(s_1) 
$
を最大化するとは限りません．しかし，上の議論から，このQ関数は

$$
\begin{aligned}
\widehat{Q}^{o, \pi}_h(s, a)
&=r^o_h(s, a) + \min\left\{b_h(s, a)+\widehat{P}_h (\cdot \mid s, a)\widehat{V}^{o, \pi}_{h+1}(\cdot), H-h\right\}\\
&= r^o_h(s, a) + \delta^{o, \widetilde{\pi}}_h(s, a) + P_h(\cdot \mid s, a) \widehat{V}^{o, \pi}_{h+1}(\cdot)\\
\widehat{Q}^{p, \pi}_h(s, a)
&=r^p_h(s, a) + \min\left\{-b_h(s, a)+\widehat{P}_h (\cdot \mid s, a)\widehat{V}^{p, \pi}_{h+1}(\cdot), H-h\right\}\\
&= r^p_h(s, a) - \delta^{p, \widetilde{\pi}}_h(s, a) + P_h(\cdot \mid s, a) \widehat{V}^{p, \pi}_{h+1}(\cdot)
\end{aligned}
$$

を満たすので，
$P$のもとで，報酬が
$$
r^o_h(s, a) + \delta^{o, \widetilde{\pi}}_h(s, a)
+ \lambda r^p_h(s, a) - \lambda \delta^{p, \widetilde{\pi}}_h(s, a)
$$
であるMDPの最適方策であることは言えます．
ただし，$\widetilde{\pi}$を事前に固定しないとダメです．
